In [1]:
import gmplot
import pandas as pd
import numpy as np
from math import radians, cos, sin, asin, sqrt

In [2]:
#Luetaan data
data = pd.read_csv('./Mittaus 2/Location.csv', delimiter=",", decimal=".")
data.head()

,time,seconds_elapsed,bearingAccuracy,speedAccuracy,verticalAccuracy,horizontalAccuracy,speed,bearing,altitude,longitude,latitude
0,1696508778679000000,2.208000,0,0,2.977515,43.896999,0.000000,0.000000,39.200001,25.513983,65.067013
1,1696508779615108000,3.144108,10,20,2.986876,26.448000,0.302750,9.921489,39.200001,25.514410,65.067186
2,1696508779942342000,3.471342,10,20,2.990148,20.299000,1.980423,326.879639,39.200001,25.514266,65.067249
3,1696508781655564800,5.184565,10,20,3.007281,16.974001,9.418491,335.661926,39.200001,25.514068,65.067432
4,1696508782427000000,5.956000,0,0,1.945684,18.065001,9.288611,335.613525,39.300003,25.514007,65.067489


In [3]:
#Määritellään funktio (Haversinen kaava), joka ottaa sisään alku- ja loppupisteen koordinaantit lon1, lat1, lon2, lat2
#ja palauttaa niiden välisen matkan
long1 = data['longitude'].iloc[0]

long2 = data['longitude'].iloc[-1]

lati1 = data['latitude'].iloc[0]

lati2 = data['latitude'].iloc[-1]



def haversine(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance in kilometers between two points  on the earth (specified in decimal degrees)
    """
    # convert decimal degrees to radians 
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])

    # haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    r = 6371 # Radius of earth in kilometers.
    return c * r #Palauttaa tämän

distancetraveled = haversine(long1, lati1, long2, lati2)*1000
print('You traveled',round(distancetraveled), 'meters')

You traveled 252 meters


In [4]:
#Lasketaan nopeus Haversinen kaavaa käyttäen (vaikka se olisi jo valmiiksi datassa)

lat = data['latitude'] #Määritellään parametri lat, joka sisältää latitudin arvot
lon = data['longitude'] #Määritellään parametri lon, joka sisältää longitudin arvot

data['dist'] = np.zeros(lat.shape[0]) #Lisätään dataan nollasarake, johon kohta lasketaan pisteiden välinen etäisyys
data['time_diff']  = np.zeros(lat.shape[0]) #Lisätään dataan nollasarake, johon kohta lasketaan pisteiden välinen aika

for i in range(lat.shape[0]-1): #For-luuppi käy läpi datan toiseksi viimeiselle riville saakka
  data['dist'][i] = haversine(lon[i], lat[i],lon[i+1], lat[i+1])*1000 #Datan pisteiden i ja i + 1 välimatka metreissä
  data['time_diff'][i] = data['seconds_elapsed'][i+1] - data['seconds_elapsed'][i]  #Peräkkäisten ajanhetkien i ja i+1 välimatka datassa


data['dist'][0] = 0
data['velocity'] = data['dist']/data['time_diff']

data.head()

C:\Users\janit\AppData\Local\Temp\ipykernel_20044\3706817946.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['dist'][i] = haversine(lon[i], lat[i],lon[i+1], lat[i+1])*1000 #Datan pisteiden i ja i + 1 välimatka metreissä
C:\Users\janit\AppData\Local\Temp\ipykernel_20044\3706817946.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['time_diff'][i] = data['seconds_elapsed'][i+1] - data['seconds_elapsed'][i]  #Peräkkäisten ajanhetkien i ja i+1 välimatka datassa
C:\Users\janit\AppData\Local\Temp\ipykernel_20044\3706817946.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from 

,time,seconds_elapsed,bearingAccuracy,speedAccuracy,verticalAccuracy,horizontalAccuracy,speed,bearing,altitude,longitude,latitude,dist,time_diff,velocity
0,1696508778679000000,2.208000,0,0,2.977515,43.896999,0.000000,0.000000,39.200001,25.513983,65.067013,0.000000,0.936108,0.000000
1,1696508779615108000,3.144108,10,20,2.986876,26.448000,0.302750,9.921489,39.200001,25.514410,65.067186,9.663841,0.327234,29.531929
2,1696508779942342000,3.471342,10,20,2.990148,20.299000,1.980423,326.879639,39.200001,25.514266,65.067249,22.426055,1.713223,13.089981
3,1696508781655564800,5.184565,10,20,3.007281,16.974001,9.418491,335.661926,39.200001,25.514068,65.067432,6.926716,0.771435,8.978997
4,1696508782427000000,5.956000,0,0,1.945684,18.065001,9.288611,335.613525,39.300003,25.514007,65.067489,1.818721,0.186678,9.742547


In [5]:
finaltime = data['seconds_elapsed'].iloc[-1]

def avgspeed(time, dist):
    hours = time / 3600
    kilometers = dist / 1000
    return kilometers/hours
    
averagespeed=avgspeed(finaltime,distancetraveled)
print('Average speed is',round(averagespeed,1),'km/h')

Average speed is 5.2 km/h


In [8]:
# Plot path in Google map

gmap = gmplot.GoogleMapPlotter(65.067013, 25.513983, 16, apikey="")

location_frame = data[['latitude','longitude']] #Luodaan Pandas data frame, jossa sarakkeina latitudi ja longitudi

location_list = location_frame.values.tolist() #Luodaan framesta lista

my_path = zip(*location_list)

gmap.plot(*my_path, color='cornflowerblue', edge_width=10)

gmap.draw('reitti2.html') #Tallennetaan tiedostoon